In [4]:
from deeppavlov import build_model

ner_model = build_model('ner_collection3_bert', download=True, install=True)


2025-09-23 17:21:53.233 INFO in 'deeppavlov.core.data.utils'['utils'] at line 97: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_coll3_torch.tar.gz to C:\Users\lexan\.deeppavlov\models\ner_rus_bert_coll3_torch.tar.gz
100%|██████████| 1.44G/1.44G [02:20<00:00, 10.2MB/s]
2025-09-23 17:24:14.675 INFO in 'deeppavlov.core.data.utils'['utils'] at line 284: Extracting C:\Users\lexan\.deeppavlov\models\ner_rus_bert_coll3_torch.tar.gz archive into C:\Users\lexan\.deeppavlov\models\ner_rus_bert_coll3_torch
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationsh

In [1]:
from deeppavlov import train_model, build_model 
from deeppavlov.core.commands.utils import parse_config

PROJECT_DIR = '..'

model_config = parse_config('ner_collection3_bert')

# dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

model_config['dataset_reader']['data_path'] = PROJECT_DIR + '/datasets/conll/'

del model_config['metadata']['download']

model_config['metadata']['variables']['MODEL_PATH'] = PROJECT_DIR + '/models'

model_config['chainer']['pipe'][1]['save_path'] = PROJECT_DIR + '/models/tag.dict'
model_config['chainer']['pipe'][1]['load_path'] = PROJECT_DIR + '/models/tag.dict'

model_config['chainer']['pipe'][2]['save_path'] = PROJECT_DIR + '/models'
model_config['chainer']['pipe'][2]['load_path'] = PROJECT_DIR + '/models'

model_config['train']['batch_size'] = 500

model_config['train']['log_every_n_batches'] = 10
model_config['train']['val_every_n_batches'] = 10



~/.deeppavlov/downloads/collection3/


In [2]:

ner_model = train_model(model_config, download=True)

2025-09-24 12:22:46.400 WARNING in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 66: TorchTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-09-24 12:22:49.182 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 104: [saving vocabulary to C:\Users\lexan\OneDrive\Documents\hackaton_lct\models\tag.dict]
Some weights of the model checkpoint at DeepPavl

{"valid": {"eval_examples_count": 4088, "metrics": {"ner_f1": 12.9292, "ner_token_f1": 17.8201}, "time_spent": "0:00:02", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 100}}


1it [00:00, 10.52it/s]

{"train": {"eval_examples_count": 500, "metrics": {"ner_f1": 56.8713, "ner_token_f1": 73.8589}, "time_spent": "0:00:04", "epochs_done": 0, "batches_seen": 10, "train_examples_seen": 5000, "loss": 1.2295032143592834}}



9it [00:00, 10.96it/s]
2025-09-24 12:22:55.270 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best ner_f1 from 12.9292 to 54.1126
2025-09-24 12:22:55.270 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2025-09-24 12:22:55.271 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 175: Saving model to C:\Users\lexan\OneDrive\Documents\hackaton_lct\models.pth.tar.


...

39it [00:12,  3.17it/s]
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

{"valid": {"eval_examples_count": 4088, "metrics": {"ner_f1": 94.3631, "ner_token_f1": 95.5484}, "time_spent": "0:00:01"}}



9it [00:00, 11.13it/s]

{"test": {"eval_examples_count": 4088, "metrics": {"ner_f1": 93.5014, "ner_token_f1": 94.8994}, "time_spent": "0:00:01"}}



Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initiali

In [10]:
model_config

{'dataset_reader': {'class_name': 'conll2003_reader',
  'data_path': '../datasets/conll/',
  'dataset_name': 'collection3',
  'provide_pos': False,
  'provide_chunk': False,
  'iobes': True},
 'dataset_iterator': {'class_name': 'data_learning_iterator'},
 'chainer': {'in': ['x'],
  'in_y': ['y'],
  'pipe': [{'class_name': 'torch_transformers_ner_preprocessor',
    'vocab_file': 'DeepPavlov/rubert-base-cased',
    'do_lower_case': False,
    'max_seq_length': 512,
    'max_subword_length': 15,
    'token_masking_prob': 0.0,
    'in': ['x'],
    'out': ['x_tokens',
     'x_subword_tokens',
     'x_subword_tok_ids',
     'startofword_markers',
     'attention_mask',
     'tokens_offsets']},
   {'id': 'tag_vocab',
    'class_name': 'simple_vocab',
    'unk_token': ['O'],
    'pad_with_zeros': True,
    'save_path': '../models/tag.dict',
    'load_path': '../models/tag.dict',
    'fit_on': ['y'],
    'in': ['y'],
    'out': ['y_ind']},
   {'class_name': 'torch_transformers_sequence_tagger',

In [3]:
from deeppavlov import build_model

ner_model = build_model(model_config, download=False)

c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNo

In [19]:
ner_model(['artfruit нектари'])

[[['artfruit', 'нектари']], [['S-BRAND', 'S-TYPE']]]